In [1]:
from google.cloud import bigquery

# Construct a BigQuery client object.
bq = bigquery.Client()
job_config = bigquery.QueryJobConfig(use_query_cache=False)

### Part 1 (County Dataset)

In [2]:
#q1 : what is the `geo_id` for Dane county?
q1 = bq.query(
"""
select geo_id from bigquery-public-data.geo_us_boundaries.counties
where county_name='Dane'
""", job_config=job_config)
q1.to_dataframe()['geo_id'][0]

'55025'

In [3]:
# Count the number of counties in the US
q = bq.query(
"""
select count(*) as num_rows 
from bigquery-public-data.geo_us_boundaries.counties
""")
q.to_dataframe()

,num_rows
0,3233


In [4]:
# Print the column names in the counties table
table_ref = bq.get_table("bigquery-public-data.geo_us_boundaries.counties")
column_names = [column.name for column in table_ref.schema]
print(column_names)

['geo_id', 'state_fips_code', 'county_fips_code', 'county_gnis_code', 'county_name', 'lsad_name', 'lsad_code', 'fips_class_code', 'mtfcc_feature_class_code', 'csa_fips_code', 'cbsa_fips_code', 'met_div_fips_code', 'functional_status', 'area_land_meters', 'area_water_meters', 'int_point_lat', 'int_point_lon', 'int_point_geom', 'county_geom']


In [5]:
#q2 : how many counties are there per state (for five states with most counties)?
q2 = bq.query(
"""
select state_fips_code, count(*) as count from bigquery-public-data.geo_us_boundaries.counties
group by state_fips_code
""", job_config=job_config)
q2_ = q2.to_dataframe().sort_values('count', ascending=False).head(5)
q2_ = q2_.set_index('state_fips_code')
q2_ = q2_.to_dict()['count']
q2_

{'48': 254, '13': 159, '51': 133, '21': 120, '29': 115}

In [6]:
#q3 : about how much should the queries for the last two questions cost?
tb = 1024**4
price_per_tb = 6.25
q3 = {}
q3['q1'] = q1.total_bytes_billed / tb * price_per_tb
q3['q2'] = q2.total_bytes_billed / tb * price_per_tb
q3

{'q1': 5.9604644775390625e-05, 'q2': 5.9604644775390625e-05}

### Part 2 : HDMA Data (Parquet in GCS)

In [7]:
#q4 : what are the datasets in your GCP project?
PROJECT_ID = "cs-544-project" # Anshu - "project-vm-398718"
DATASET_ID = "p8"
PARQUET_LINK = "gs://cs544_p8_data4lyf/hdma-wi-2021.parquet" # Anshu - "gs://cs544_p8_data4lyf2/hdma-wi-2021.parquet"

ds = bigquery.Dataset(f"{PROJECT_ID}.{DATASET_ID}")
bq.create_dataset(ds, exists_ok=True)

config = bigquery.LoadJobConfig(source_format="PARQUET", write_disposition="WRITE_TRUNCATE")
bq.load_table_from_uri(PARQUET_LINK, f"{PROJECT_ID}.{DATASET_ID}.hdma", job_config=config)

[ds.dataset_id for ds in bq.list_datasets(PROJECT_ID)]

['awesds', 'awesomeds', 'p8']

In [8]:
#q5 : how many loan applications are there in the HDMA data for each county?
import pandas as pd
h1 = bq.query(
"""
select county_code, count(*) as count
from p8.hdma
group by county_code
""")
h1 = h1.to_dataframe()

h2 = bq.query(
"""
select distinct county_fips_code as county_code, county_name
from bigquery-public-data.geo_us_boundaries.counties
""")
h2 = h2.to_dataframe()
q5_ = pd.merge(h1, h2, on='county_code', how='inner').sort_values('count', ascending=False).head(10)
q5_ = q5_.set_index('county_name')
q5 = q5_.to_dict()['count']
q5

{'Milwaukee': 46570,
 'Dane': 38557,
 'Waukesha': 34159,
 'Brown': 15615,
 'Racine': 13007,
 'Outagamie': 11523,
 'Kenosha': 10744,
 'Washington': 10726,
 'Rock': 9834,
 'Winnebago': 9310}

### Part 3: Application Data (Google Sheet Linked to Form)

In [9]:
#q6 : how many applications are there with your chosen income?

url = "https://docs.google.com/spreadsheets/d/11UeIBqQylAyNUBsIO54p6WiYJWHayQMfHDbUWq1jGco/"

external_config = bigquery.ExternalConfig("GOOGLE_SHEETS")
external_config.source_uris = [url]
external_config.options.skip_leading_rows = 1
external_config.autodetect = True

table = bigquery.Table("cs-544-project.p8.applications")
table.external_data_configuration = external_config

table = bq.create_table(table, exists_ok=True)

# for t in bq.list_tables("cs-544-project.p8"):
#     print(t.table_id)

q = bq.query(
"""
select * from cs-544-project.p8.applications
where income = 300000
""")
q.to_dataframe().shape[0]

7

In [10]:
#q7 : how many applications are there in the Google sheet per WI county?
q = bq.query(
"""
SELECT c.county_name AS county_name, COUNT(*) AS count
FROM `bigquery-public-data.geo_us_boundaries.counties` AS c
JOIN `cs-544-project.p8.applications` AS a
ON ST_CONTAINS(c.county_geom, ST_GEOGPOINT(a.longitude, a.latitude))
WHERE c.state_fips_code = '55'
GROUP BY county_name
""")
result = q.to_dataframe()
applications_per_county = dict(zip(result['county_name'], result['count']))
applications_per_county

{'Sheboygan': 7,
 'Dane': 87,
 'Door': 21,
 'Jefferson': 3,
 'Ozaukee': 2,
 'Columbia': 6,
 'Walworth': 4,
 'Marinette': 3,
 'Winnebago': 3,
 'Iowa': 1,
 'Ashland': 2,
 'Brown': 7,
 'Eau Claire': 1,
 'Bayfield': 5,
 'Milwaukee': 6,
 'Barron': 2,
 'Wood': 2,
 'Manitowoc': 2,
 'Oneida': 4,
 'Fond du Lac': 1,
 'Marathon': 1,
 'Dodge': 1,
 'Polk': 1,
 'Sauk': 3,
 'Chippewa': 1,
 'Kenosha': 1,
 'Outagamie': 2,
 'Kewaunee': 3,
 'Monroe': 2,
 'Crawford': 1,
 'Vilas': 1,
 'Green Lake': 2,
 'St. Croix': 1,
 'Adams': 1,
 'Douglas': 1,
 'Price': 1}

### Part 4: Machine Learning

In [11]:
%load_ext google.cloud.bigquery

In [12]:
%%bigquery
CREATE OR REPLACE MODEL `cs-544-project.p8.m1`
OPTIONS(model_type='linear_reg', input_label_cols=['loan_amount']) 

AS

SELECT loan_amount, income, loan_term
FROM `cs-544-project.p8.hdma`

Query is running:   0%|          |

""


In [13]:
import time
while True:
    model_present = False
    for model in bq.list_models("cs-544-project.p8"):
        if model.model_id == "m1":
            model_present = True
            break
    if model_present:
        break
    time.sleep(5)

In [14]:

%%bigquery eval_df 
SELECT * FROM ML.EVALUATE(MODEL `cs-544-project.p8.m1`, (
    SELECT loan_amount, income, loan_term
    FROM `cs-544-project.p8.hdma`
))

Query is running:   0%|          |

Downloading:   0%|          |

In [15]:
eval_df

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,77104.735568,1.286932e+10,0.429716,55477.208425,0.291653,0.291653


In [16]:
#q8
eval_df["r2_score"][0]

0.29165346922391866

In [17]:
%%bigquery weights_df
SELECT *
FROM ML.WEIGHTS(MODEL `cs-544-project.p8.m1`)

Query is running:   0%|          |

Downloading:   0%|          |

In [18]:
weights_df

,processed_input,weight,category_weights
0,income,0.804631,[]
1,loan_term,380.225284,[]
2,__INTERCEPT__,-579.734123,[]


In [19]:
#q9
income_weight = weights_df.loc[weights_df['processed_input'] == 'income', 'weight'].values[0]
income_weight

0.8046310253990151

In [20]:
%%bigquery appl_df 
SELECT * FROM `cs-544-project.p8.applications`

Query is running:   0%|          |

Downloading:   0%|          |

In [21]:
appl_df

,Timestamp,loan_amount,income,latitude,longitude
0,2023-11-27 21:05:40.625000+00:00,100000.0,10000.0,43.069646,-89.753298
1,2023-11-27 21:10:35.948000+00:00,200000.0,20000.0,45.058784,-87.665404
2,2023-11-27 21:11:51.383000+00:00,200000.0,20000.0,45.000711,-87.229077
3,2023-11-27 21:13:07.711000+00:00,300000.0,30000.0,44.523038,-88.072896
4,2023-11-27 21:14:15.019000+00:00,200000.0,30000.0,42.528472,-88.602056
...,...,...,...,...,...
309,2023-12-13 22:13:28.676000+00:00,50000.0,100000.0,43.373119,-89.579532
310,2023-12-13 22:16:47.974000+00:00,400000.0,250000.0,43.068826,-89.408038
311,2023-12-13 22:22:09.481000+00:00,650000.0,115000.0,42.844444,89.009083
312,2023-12-13 22:46:43.207000+00:00,100.0,100.0,0.000000,0.000000


In [22]:
#q10
loan_term_weight = weights_df.loc[weights_df['processed_input'] == 'loan_term', 'weight'].values[0]
intercept = weights_df.loc[weights_df['processed_input'] == '__INTERCEPT__', 'weight'].values[0]

# Make the prediction in applications table
prediction = income_weight * appl_df["income"] + loan_term_weight * 360 + intercept
ask_ratio = (appl_df["loan_amount"] > prediction).sum() / appl_df.shape[0]
ask_ratio

0.5859872611464968